# Brain teaser

[Original Github REPO](https://github.com/1171-jpg/BrainTeaser)

[Leaderboard](https://huggingface.co/spaces/HuggingFaceH4/open_llm_leaderboard)


In [2]:
!pip install -q -U langchain transformers bitsandbytes accelerate

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 23.8.0 requires cubinlinker, which is not installed.
cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
cudf 23.8.0 requires ptxcompiler, which is not installed.
cuml 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
dask-cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
keras-cv 0.8.2 requires keras-core, which is not installed.
keras-nlp 0.9.3 requires keras-core, which is not installed.
tensorflow-decision-forests 1.8.1 requires wurlitzer, which is not installed.
apache-beam 2.46.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.8 which is incompatible.
apache-beam 2.46.0 requires numpy<1.25.0,>=1.14.3, but you have numpy 1.26.4 which is incompatible.
apache-beam 2.46.0 requires pyarrow<10.0.0,>=3.0.0, but you have pyarrow 15.0.2 which is incompatible.
cudf

In [3]:
import numpy as np
import json
from datasets import load_dataset
from random import shuffle
import random

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

import torch
from torch.nn.functional import normalize
from transformers import AutoModel, AutoTokenizer

from tqdm.auto import tqdm
import numpy as np
import os
from transformers import StoppingCriteriaList

from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline
import torch

2024-06-04 18:46:32.700722: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-04 18:46:32.700821: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-04 18:46:32.810012: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
# Ako imamo vise ponudenih odgovora od 4, da postupak bude automatski u svim funkcijama
letters = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P'][:4]
letters, len(letters)

(['A', 'B', 'C', 'D'], 4)

## Metrike

Preuzeto direktno s njihovog repoa

In [5]:
import json
from tqdm import tqdm
import torch
import logging
import argparse
import numpy as np

def getResultdata(result_data):
    choice_to_index = {letters[i]: i for i in range(len(letters))}
    choice_to_index[None] = len(letters)

    word_play = {}
    reverse_play = {}
    for item in result_data:
        item_type = item['id'].split("-")[0]
        item_id = item['id'].split("-")[1].split("_")[0]
        if item_type == 'WP':
            if item_id not in word_play:
                word_play[item_id] = [0,0,0]
        else:
            if item_id not in reverse_play:
                reverse_play[item_id] = [0,0,0]

    for item in result_data:
        item_type = item['id'].split("-")[0]
        item_id = item['id'].split("-")[1].split("_")[0]
        ad_type = 0
        if 'SR' in item['id']:
            ad_type = 1
        elif 'CR' in item['id']:
            ad_type = 2
        else:
            ad_type = 0

        if item_type == 'WP':
            if choice_to_index[item['predict']] == item['label']:
                word_play[item_id][ad_type] = 1
        else:
            if choice_to_index[item['predict']] == item['label']:
                reverse_play[item_id][ad_type] = 1
                
    return word_play,reverse_play


def getMetric(data_list):
    data_list = np.array(data_list)
    overall_accuracy = np.sum(data_list)/3/len(data_list)
    original_accuracy = np.sum(data_list,axis = 0)[0]/len(data_list)
    semantic_accuracy = np.sum(data_list,axis = 0)[1]/len(data_list)
    context_accuracy = np.sum(data_list,axis = 0)[2]/len(data_list)
    ori_sema = np.sum([1 if item[0]==1 and item[1] == 1 else 0 for item in data_list])/len(data_list)
    ori_sema_cont = np.sum([1 if item[0]==1 and item[1] == 1 and item[2] == 1  else 0 for item in data_list])/len(data_list)
    
    print("over_all accuracy {}".format(overall_accuracy))
    print("single_original_accuracy {}".format(original_accuracy))
    print("single_semantic_accuracy {}".format(semantic_accuracy))
    print("single_context_accuracy {}".format(context_accuracy))
    print("sr_accuracy {}".format(ori_sema))
    print("cr_accuracy {}".format(ori_sema_cont))

    return {'over_all accuracy':overall_accuracy,'original_accuracy':original_accuracy,'semantic_accuracy':semantic_accuracy,'context_accuracy':context_accuracy,'ori_sema':ori_sema,'ori_sema_cont':ori_sema_cont}


def getSeperateResult(word_play,reverse_thinking):
    final_result = {}
    word_data_list = []
    word_data_list = list(word_play.values())
    print('#########Wordplay##########')
    final_result['wordplay'] = getMetric(word_data_list)
    
    reverse_data_list = []
    for item in reverse_thinking.values():
        reverse_data_list.append(item)
    print('#########Sentence##########')   
    final_result['sentence'] = getMetric(reverse_data_list)  
    
    
    all_data = word_data_list + reverse_data_list
    print('#########All data##########') 
    final_result['all'] = getMetric(all_data) 
    
    return final_result

### Spremanje datoteka

Automatski dodaje +1 na naziv ako vec postoji takva datoteka

Uzima format model_name + name + redni broj za datoteku

Unutra pise prvi testni primjer (example) i rezultate

In [6]:
# def save(final_result, name=None, few_shot=False, example=format):
#     try:
#         os.mkdir('results')
#     except:
#         pass
#     model_name = model_path.split('/')[-1]
#     inserted_name = "" if name is None else "_" + name
#     i = 1
#     while True:
#         file_path = f'results/{model_name}{inserted_name}_{i}.txt'
#         try:
#             with open(file_path, 'r'):
#                 i += 1
#         except:
#             with open(file_path, 'w') as file:
#                 file.write(model_name+'\n')
#                 if few_shot:
#                     file.write(few_shot + '\n')
#                 if not isinstance(example, str):
#                     example = json.dumps(example)
#                 file.write(example + '\n')
#                 file.write(json.dumps(final_result, indent=4))
#                 return

dirname= 'resultsnew'
pickle_dir = 'pickled_results'

def create_dir(name):
    try:
        os.mkdir(name)
    except:
        pass

def save(final_result, name=None, few_shot=False, example=format, data=None):
    create_dir(dirname)
    create_dir(pickle_dir)
    
    model_name = model_path.split('/')[-1]
    inserted_name = "" if name is None else "_" + name
    i = 1
    while True:
        file_path = f'{dirname}/{model_name}{inserted_name}_{i}.txt'
        pickle_path = f'{pickle_dir}/{model_name}{inserted_name}_{i}.json'
        try:
            with open(file_path, 'r'):
                i += 1
        except:
            with open(file_path, 'w') as file:
                file.write(model_name+'\n')
                if few_shot:
                    file.write(few_shot + '\n')
                if not isinstance(example, str):
                    example = json.dumps(example)
                file.write(example + '\n')
                file.write(json.dumps(final_result, indent=4))
            if data is None:
                print(f"Warning, {model_name} results data not saving in json, please provide data arg")
            else:
                with open(pickle_path, 'w') as file:
                    file.write(json.dumps(data, indent=4))
            return


### Load the model

Login kako bi se mogli ucitati napredniji modeli

In [7]:
# device = 'cuda' if torch.cuda.is_available() else 'cpu'
device = "cuda:0" if torch.cuda.is_available() else 'cpu'
print(device)


cuda:0


Logiranje u huggingiface
treba kljuc, njega nabavite na huggingface stranici

In [8]:
from huggingface_hub import login
login()

BitsAndBytesConfig je novi, to kao kvantizira model pa stane na graficku

In [9]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AutoModelForCausalLM
# model_path = 'google/flan-t5-xxl'
# model_path = 'google/flan-t5-xl'
# model_path = 'google/flan-t5-large'
# model_path = 'mistralai/Mistral-7B-Instruct-v0.2'
# model_path = 'microsoft/phi-2'
model_path = 'microsoft/Phi-3-mini-128k-instruct'
# model_path = 'microsoft/Phi-3-mini-4k-instruct'
# model_path = '01-ai/Yi-1.5-6B'
# model_path = 'netcat420/MFANN3bv0.6'
# model_path = '01-ai/Yi-1.5-9B-Chat'
# model_path = 'google/gemma-1.1-7b-it'
# model_path = 'meta-llama/Meta-Llama-3-8B-Instruct'
# model_path = 'tiiuae/falcon-7b-instruct'

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

tokenizer = AutoTokenizer.from_pretrained(model_path)

if model_path.find('flan') >= 0:
    model = AutoModelForSeq2SeqLM.from_pretrained(
        model_path, 
        trust_remote_code=True,
        device_map = 'cuda'
    )
# elif model_path.find('mistral') >= 0:
#     model = AutoModelForCausalLM.from_pretrained(
#         model_path,
#         device_map="auto",
#         quantization_config=quantization_config,
#         trust_remote_code=True,
#     )
else:
#     model = AutoModelForCausalLM.from_pretrained(
#         model_path,
#         trust_remote_code=True,
#         device_map = 'cuda'
#     )
    model = AutoModelForCausalLM.from_pretrained(
        model_path,
#         device_map="auto",
        device_map="cuda:0",
        quantization_config=quantization_config,
        trust_remote_code=True,
    )

tokenizer_config.json:   0%|          | 0.00/3.17k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/568 [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/16.3k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.67G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/172 [00:00<?, ?B/s]

# Format pitanja
Format za mystral, za ostale modele maknuti linije `<s>[INST]`

In [82]:
newline = '\n'

# if model_path.find('mistral') >= 0:
#     format = f"""
# <s>[INST]
# Question: {"{}"}
# Choice:
# {''.join('(' + x + ') {}' + newline for x in letters)}
# [/INST]
# Answer:("""
    
# else:
format = f"""Question: {"{}"}

Choices:
{''.join('(' + x + ') {}' + newline for x in letters)}
Answer:("""

demonstration = "The following question is a brainteaser."
format = demonstration + "\n\n" + format

print(format)

The following question is a brainteaser.

Question: {}

Choices:
(A) {}
(B) {}
(C) {}
(D) {}

Answer:(


# Dataset

- Skinuti s [linka](https://drive.google.com/drive/u/0/folders/1kiFXp5fqpf8--NQJJAlBIBpfSaXTk1UY)

- Staviti u folder `/data`, ako na Kaggleu, onda napraviti svoj dataset te ga dodati pod input

In [83]:
def load_dataset():
    # few_shot_examples = list(np.load("./{dataset_path}/data/demonstration.npy",allow_pickle=True))
    data_path = "/kaggle/input/tar-project-data" # - Josip
    # data_path = '/kaggle/input/brainteaser/data' # - Goran
    sentence_data_path = f"{data_path}/SP-train.npy"
    wordplay_data_list = f"{data_path}/WP-train.npy"
    sentence_data_list = list(np.load(sentence_data_path,allow_pickle=True))
    wordplay_data_list = list(np.load(wordplay_data_list,allow_pickle=True))

    test_data_list = sentence_data_list + wordplay_data_list
    print(f"Dataset length {len(test_data_list)}")
    
    return test_data_list

test_data_list = load_dataset()
output_name = "normal"

Dataset length 903


In [84]:
def get_single_demo(sample):
    sample_demo = format.format(
        sample['question'], *sample['choice_list'])
    return sample_demo

good_responses = [f'{x})' for x in letters]
print("good_responses", good_responses)

good_responses ['A)', 'B)', 'C)', 'D)']


## Generiranje dataseta

### Pomocne funkcije

In [85]:
def set_predictions(data_list):
    """Uzima response i parsira ga tako da pod predict stavi slovo koje je napisano"""
    for index,item in enumerate(data_list):
        item['predict'] = None
        for x in letters:
            if (f'{x})') in item['response']:
                item['predict'] = x

        if item['predict'] is None:
            print(index)

def custom_stopping_criteria(input_ids: torch.LongTensor, score: torch.FloatTensor, **kwargs) -> bool:
    """
    Funkcija kako bi se dinamicki prepoznalo kad treba prestati generirati tekst:
    ne nakon fiksnog broja tokena, vec kad model napise rjesenje. Najcesce
    ce to ipak biti medu prva dva tokena
    """
    decoded = tokenizer.decode(input_ids[0][-3:])
    for good_response in good_responses:
        if good_response in decoded:
            return True
    return False

stopping_criteria = StoppingCriteriaList([custom_stopping_criteria])

In [86]:
def generate(samples, tokens=10, all_tokens=True, few_shot=False):
    for sample in tqdm(samples):
        if few_shot:
            text = demonstration + get_single_demo(sample)
        else:
            text = get_single_demo(sample)
        inputs = tokenizer.encode(text, return_tensors="pt", return_attention_mask=False).to(device)
        original_tokens = len(inputs[0])
        outputs = model.generate(
            inputs,
            pad_token_id=tokenizer.eos_token_id,
            do_sample = False,
            max_new_tokens=tokens,
            stopping_criteria=stopping_criteria
        )
        outputs = outputs[0][0 if all_tokens else original_tokens:]
        sample['response'] = tokenizer.decode(outputs)
    set_predictions(samples)

In [87]:
def create_with_question(i):
    """Generira sample tako da zamijeni drugi distraction s pitanjem"""
    current = test_data_list[i]
    new = {**current}
    choice_list = [*current['choice_list']]
    choice_list[current['choice_order'].index(2)] = current['question'][:-1]
    new['choice_list'] = choice_list
    return new

def test_with_question(generated_data_list):
    """Testira radi li generiranje ispravno"""
    for i, sample in enumerate(generated_data_list):
        if i in [227, 755, 714, 715, 716]: continue
        assert sample['distractor2'] not in sample['choice_list'], i
        assert sample['question'][:-1] in sample['choice_list'], i
        assert str(sample['answer']) in sample['choice_list'], i
        assert str(sample['distractor1']) in sample['choice_list'], i

def count_with_question(generated_data_list):
    """Broji koliko puta je model predvidio da je pitanje odgovor na pitanje"""
    total = 0
    for sample in generated_data_list:
        if sample['choice_list'][letters.index(sample['predict'])] == sample['question'][:-1]:
            total += 1
    return total, len(generated_data_list)

def create_with_ordered_correct_answer(i, target_position):
    """Generira sample tako da je odgovor uvijek na target position"""
    current = test_data_list[i]
    new = {**current}
    choice_list = [*current['choice_list']]
    correct_answer = choice_list[current["label"]]
    choice_list.remove(correct_answer)
    choice_list.insert(target_position,correct_answer)
    new['choice_list'] = choice_list
    new['label'] = target_position
    return new

def test_with_ordered_correct_answer(generated_data_list):
    """Testira radi li generiranje ispravno"""
    for original, new in zip(test_data_list, generated_data_list):
        assert set(original['choice_list']) == set(new['choice_list']), str(original) + '\n'+ str(new)
        assert new['choice_list'][new['label']] == original['choice_list'][original['label']]

def create_with_k_answers(i, k):
    """Generira sample tako da doda kao dodatne odgovore distractore iz drugih pitanja"""
    current = test_data_list[i]
    choice_list = [*current['choice_list']]
    choice_order = [*current['choice_order']]
    while len(choice_list) < k-1:
        i -= 11
        previous = test_data_list[i]
        choice_list.insert(-1, previous['distractor1'])
        choice_list.insert(-1, previous['distractor2'])
    last_order = choice_list[-1]
    choice_list = choice_list[:-1]
#     print(current['answer'])
    shuffle(choice_list)
    choice_list.append(last_order)
    try:
        label = choice_list.index(current['answer'])
    except Exception as e:
        print(current['answer'])
        label = len(choice_list)-1
    new = {**current}
    new['label'] = label
    new['choice_list'] = choice_list
    return new

def test_with_k_answers(generated_data_list):
    """Testira radi li generiranje ispravno"""
    for original, new in zip(test_data_list, generated_data_list):
        assert set(original['choice_list']).issubset(set(new['choice_list'])), str(original) + '\n'+ str(new)

In [88]:
random.seed(10)
generated_data_list = []
for i in range(len(test_data_list)):
#     new = create_with_k_answers(i, len(letters))
    new = create_with_ordered_correct_answer(i, 0)
#     new = create_with_question(i)
    generated_data_list.append(new)
# test_with_question(generated_data_list)
test_with_ordered_correct_answer(generated_data_list)
test_with_k_answers(generated_data_list)
generated_data_list[0]

{'id': 'SP-0',
 'question': 'Mr. and Mrs. Mustard have six daughters and each daughter has one brother. But there are only 9 people in the family, how is that possible?',
 'answer': 'Each daughter shares the same brother.',
 'distractor1': 'Some daughters get married and have their own family.',
 'distractor2': 'Some brothers were not loved by family and moved away.',
 'distractor(unsure)': 'None of above.',
 'label': 0,
 'choice_list': ['Each daughter shares the same brother.',
  'Some daughters get married and have their own family.',
  'Some brothers were not loved by family and moved away.',
  'None of above.'],
 'choice_order': [1, 0, 2, 3]}

In [89]:
print(get_single_demo(test_data_list[0]))

The following question is a brainteaser.

Question: Mr. and Mrs. Mustard have six daughters and each daughter has one brother. But there are only 9 people in the family, how is that possible?

Choices:
(A) Some daughters get married and have their own family.
(B) Each daughter shares the same brother.
(C) Some brothers were not loved by family and moved away.
(D) None of above.

Answer:(


# Netocni odgovori u gibberish
pretvara sve netocne odgovore u gibberish

In [90]:
def turn_to_gibberish():
    output_name = "gibberish_all"
    
    test_data_list = load_dataset()

    random.seed(10)
    for dict in test_data_list:
        choice_list = dict["choice_list"]
        for i in range(len(choice_list)):
            if i != dict["label"]:
                s = choice_list[i]
                choice_list[i] = ''.join(random.sample(s,len(s)))
        
        if dict["id"] == "SP-88":
            for key,item in dict.items():
                print(f"{key}: {item}")
    
    return test_data_list, output_name

# Netocni odgovori u gibberish, osim None of the above
pretvara sve netocne odgovore u gibberish, osim odgovora none of the above

In [91]:
def gibberish_keep_none_of_above():
    output_name = "gibberish_keep_none_of_above"
    
    test_data_list = load_dataset()
    
    random.seed(10)
    for dict in test_data_list:
#         print(dict["question"])
        distractor1_index = dict["choice_order"].index(1)
        choice_list = dict["choice_list"]
        d1 = choice_list[distractor1_index]
        dict["distractor1"] = ''.join(random.sample(d1,len(d1)))
        choice_list[distractor1_index] = dict["distractor1"]

        
        distractor2_index = dict["choice_order"].index(2)
        choice_list = dict["choice_list"]
        d2 = choice_list[distractor2_index]
        dict["distractor2"] = ''.join(random.sample(d2,len(d2)))
        choice_list[distractor2_index] = dict["distractor2"]
    
    return test_data_list, output_name

# Correct answer umjesto netočnog odgovora
Zamijenjuje prvi netocan odgovor s "Correct answer.", da bi se zamijenilo vise odgovore moze se maknuti break.

In [92]:
def replace_with_correct_answer():
    output_name = "replace_with_correct_answer"
    
    test_data_list = load_dataset()

    for dict in test_data_list:
        choice_list = dict["choice_list"]
        for i in range(len(choice_list)):
            if i != dict["label"]:
                s = choice_list[i]
                choice_list[i] = "Correct answer."
#                 break # makni ovo da se svi netocni odgovori zamijene
    
    return test_data_list, output_name

# Wrong answer umjesto netočnog odgovora
Zamijenjuje prvi netocan odgovor s "Wrong answer.", da bi se zamijenilo vise odgovore moze se maknuti break.

In [93]:
def replace_with_wrong_answer():
    output_name = "replace_with_wrong_answer"
    
    test_data_list = load_dataset()

    for dict in test_data_list:
        choice_list = dict["choice_list"]
        for i in range(len(choice_list)):
            if i != dict["label"]:
                s = choice_list[i]
                choice_list[i] = "Wrong answer."
#                 break # makni ovo da se svi netocni odgovori zamijene
    
    return test_data_list, output_name

# Označi točan odgovor
Stavlja oznaku kod točnog odgovora da je on točan.

In [94]:
def label_correct_answer():
    output_name = "label_correct_answer"
    
    test_data_list = load_dataset()

    for dict in test_data_list:
        choice_list = dict["choice_list"]
        for i in range(len(choice_list)):
            if i == dict["label"]:
                s = choice_list[i]
                choice_list[i] = s + " (correct answer)"
#                 break # makni ovo da se svi netocni odgovori zamijene
    
    return test_data_list, output_name

# Označi netočan odgovor kao točan
Stavlja oznaku kod prvog netočnog odgovora da je on točan.

In [95]:
def label_incorrect_as_correct_answer():
    output_name = "label_incorrect_as_correct_answer"
    
    test_data_list = load_dataset()

    for dict in test_data_list:
        choice_list = dict["choice_list"]
        for i in range(len(choice_list)):
            if i != dict["label"]:
                s = choice_list[i]
                choice_list[i] = s + " (correct answer)"
                break # makni ovo da se svi netocni odgovori zamijene
    
    return test_data_list, output_name

# Ponudi odgovor "I don't know.".
Dodaj "I don't know." umjesto prvog pogresnog odgovora.

In [96]:
def add_i_dont_know():
    output_name = "add_i_dont_know"
    
    test_data_list = load_dataset()

    for dict in test_data_list:
        choice_list = dict["choice_list"]
        for i in range(len(choice_list)):
            if i != dict["label"]:
                s = choice_list[i]
                choice_list[i] = "I don't know."
                break # makni ovo da se svi netocni odgovori zamijene
    
    return test_data_list, output_name

# Istinita tvrdnja umjesto netočnog odgovora
Zamijenjuje prvi netocan odgovor s nekom istinitom tvrdnjok kao npr "Grass is green.", da bi se zamijenilo vise odgovore moze se maknuti break.

In [97]:
def replace_with_true_statement():
    output_name = "replace_with_true_statement"
    
    test_data_list = load_dataset()

    for dict in test_data_list:
        choice_list = dict["choice_list"]
        for i in range(len(choice_list)):
            if i != dict["label"]:
                s = choice_list[i]
                choice_list[i] = "Grass is green."
                break # makni ovo da se svi netocni odgovori zamijene
    
    return test_data_list, output_name

# Zamijeni distraktor 1 s ponovljenim pitanjem

In [98]:
def replace_distractor_with_question():
    output_name = "replace_distractor_with_question"
    
    test_data_list = load_dataset()

    for dict in test_data_list:
        dict["distractor1"] = dict["question"]
        distractor1_index = dict["choice_order"].index(1)
        choice_list = dict["choice_list"]
        choice_list[distractor1_index] = dict["question"]
    
    return test_data_list, output_name

In [117]:
# test_data_list, output_name = turn_to_gibberish()
# test_data_list, output_name = gibberish_keep_none_of_above()
# test_data_list, output_name = replace_with_correct_answer()
# test_data_list, output_name = replace_with_wrong_answer()
# test_data_list, output_name = label_correct_answer()
# test_data_list, output_name = label_incorrect_as_correct_answer()
# test_data_list, output_name = add_i_dont_know()
# test_data_list, output_name = replace_with_true_statement()
test_data_list, output_name = replace_distractor_with_question()
print(get_single_demo(test_data_list[0]))

Dataset length 903
The following question is a brainteaser.

Question: Mr. and Mrs. Mustard have six daughters and each daughter has one brother. But there are only 9 people in the family, how is that possible?

Choices:
(A) Mr. and Mrs. Mustard have six daughters and each daughter has one brother. But there are only 9 people in the family, how is that possible?
(B) Each daughter shares the same brother.
(C) Some brothers were not loved by family and moved away.
(D) None of above.

Answer:(


## Testiranje

In [118]:
generate(test_data_list[:10], tokens=1000, all_tokens=False, few_shot=False)

100%|██████████| 10/10 [00:02<00:00,  3.86it/s]


In [119]:
for sample in test_data_list[:10]:
    print("\n>", sample['response'])


> B)

> C)

> B)

> C)

> B)

> A)

> C)

> C)

> C)

> D)


In [120]:
generate(test_data_list, tokens=20, all_tokens=False, few_shot=False)

100%|██████████| 903/903 [03:55<00:00,  3.84it/s]


In [121]:
word_play,sentence_play = getResultdata(test_data_list)
# word_play,sentence_play = getResultdata(test_data_list)
final_result = getSeperateResult(word_play, sentence_play)

#########Wordplay##########
over_all accuracy 0.44191919191919193
single_original_accuracy 0.44696969696969696
single_semantic_accuracy 0.4393939393939394
single_context_accuracy 0.4393939393939394
sr_accuracy 0.29545454545454547
cr_accuracy 0.13636363636363635
#########Sentence##########
over_all accuracy 0.5167652859960552
single_original_accuracy 0.5207100591715976
single_semantic_accuracy 0.47337278106508873
single_context_accuracy 0.5562130177514792
sr_accuracy 0.40236686390532544
cr_accuracy 0.24260355029585798
#########All data##########
over_all accuracy 0.4839424141749723
single_original_accuracy 0.4883720930232558
single_semantic_accuracy 0.4584717607973422
single_context_accuracy 0.5049833887043189
sr_accuracy 0.3554817275747508
cr_accuracy 0.19601328903654486


In [122]:
save(final_result, name=output_name, example=get_single_demo(test_data_list[0]))

Warning, Phi-3-mini-128k-instruct results data not saving in json, please provide data arg


In [ ]:
cudni_odgovori = [86,103,107,144,169,200,242,303,363,366,409,414,424,444,530,531,550,551,562,566,567,578,588,597,599,604,609,610,611,613,621,622,642,643,663,675,691,722,725,730,739,746,758,759,760,771,794,795,797,805,806,822,842,874,878,900]
for i in cudni_odgovori:
    print(get_single_demo(test_data_list[i]))
    print(test_data_list[i]["response"])

In [ ]:
sum([int(test_data_list[i]['label']==1) for i in range(102)])

In [84]:
print(get_single_demo(test_data_list[86]))
test_data_list[86]["response"]

Question: A man accidentally stumbled and fell into a 1-meter-deep pool. He couldn't swim at all, and there were no other people around him, but he was fine at the end. Why?
Choice:
(A) 
He can wait until the water evaporates completely.
(B) He can just stand up as the pool is only 1 meter deep.
(C) He can wait until water level is high enough, he can safely step out of the pool. 
(D) None of above.

Answer:(


'B)'

In [ ]:
def eval_with_correct_fixed(j):
    random.seed(10)
    generated_data_list = []
    for i in range(len(test_data_list)):
        new = create_with_ordered_correct_answer(i, j)
        generated_data_list.append(new)
    test_with_ordered_correct_answer(generated_data_list)
    
    print(get_single_demo(generated_data_list[0]))
    
    generate(generated_data_list, tokens=20, all_tokens=False, few_shot=False)
    
    word_play,sentence_play = getResultdata(generated_data_list)
    final_result = getSeperateResult(word_play, sentence_play)

    save(final_result, name=f"always-{j}", example=get_single_demo(generated_data_list[0]))

In [ ]:
eval_with_correct_fixed(3)
eval_with_correct_fixed(1)
eval_with_correct_fixed(2)
eval_with_correct_fixed(0)

In [98]:
import copy

pom = copy.deepcopy(test_data_list[0])
pom["distractor1"] = pom["question"]
distractor1_index = pom["choice_order"].index(1)
choice_list = pom["choice_list"]
choice_list[distractor1_index] = pom["question"]
pom

{'id': 'SP-0',
 'question': 'Mr. and Mrs. Mustard have six daughters and each daughter has one brother. But there are only 9 people in the family, how is that possible?',
 'answer': 'Each daughter shares the same brother.',
 'distractor1': 'Mr. and Mrs. Mustard have six daughters and each daughter has one brother. But there are only 9 people in the family, how is that possible?',
 'distractor2': 'Some brothers were not loved by family and moved away.',
 'distractor(unsure)': 'None of above.',
 'label': 1,
 'choice_list': ['Mr. and Mrs. Mustard have six daughters and each daughter has one brother. But there are only 9 people in the family, how is that possible?',
  'Each daughter shares the same brother.',
  'Some brothers were not loved by family and moved away.',
  'None of above.'],
 'choice_order': [1, 0, 2, 3],
 'response': 'B)',
 'predict': 'B'}

In [99]:
for key, item in test_data_list[0].items():
    print(f"{key}: {item}")

print()
print(get_single_demo(test_data_list[0]))
print()
print(get_single_demo(pom))

id: SP-0
question: Mr. and Mrs. Mustard have six daughters and each daughter has one brother. But there are only 9 people in the family, how is that possible?
answer: Each daughter shares the same brother.
distractor1: Some daughters get married and have their own family.
distractor2: Some brothers were not loved by family and moved away.
distractor(unsure): None of above.
label: 1
choice_list: ['Some daughters get married and have their own family.', 'Each daughter shares the same brother.', 'Some brothers were not loved by family and moved away.', 'None of above.']
choice_order: [1, 0, 2, 3]
response: B)
predict: B

Question: Mr. and Mrs. Mustard have six daughters and each daughter has one brother. But there are only 9 people in the family, how is that possible?
Choice:
(A) Some daughters get married and have their own family.
(B) Each daughter shares the same brother.
(C) Some brothers were not loved by family and moved away.
(D) None of above.

Answer:(

Question: Mr. and Mrs. Mus

In [ ]:
# pip install accelerate